In [1]:
import pandas as pd
import numpy as np
import warnings

import statsmodels.api as sm

from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures

pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
#pd.options.display.float_format = '{:.2f}'.format
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn

data = '/kaggle/input/home-credit-credit-risk-model-stability/csv_files/'

In [2]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64',
                'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
                else:
                    if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df[col] = df[col].astype(np.float32)
                    else:
                        df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'
                      .format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [3]:
def set_table_dtypes(df: pd.DataFrame) -> pd.DataFrame:
    for col in df.columns:
        # Last letter of column name will help you determine the type
        if col[-1] in ("P", "A"):
            df[col] = df[col].astype(float)
    return df

In [4]:
def convert_strings(df: pd.DataFrame) -> pd.DataFrame:
    for col in df.columns:
        # Check if the column's data type is 'object' or 'string'
        if df[col].dtype == 'object' or df[col].dtype == 'string':
            # Convert column to 'category' data type
            df[col] = df[col].astype('category')
            # Get current categories and add 'Unknown'
            new_categories = df[col].cat.categories.tolist() + ['Unknown']
            # Define new categorical data type with 'Unknown'
            df[col] = df[col].cat.set_categories(new_categories, ordered=True)
    return df

In [5]:
def preprocess_data(data, imputer):
    data = imputer.transform(data)
    return data

In [6]:
def train_model_statsmodels(X, y):
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit(cov_type='HC0')
    return model

In [7]:
def create_k_folds(df, fold_n=5):
    folds = {}
    fold_size = len(df) // fold_n
    for i in range(fold_n):
        start = i * fold_size
        if i == fold_n - 1:  # In the last fold, include all remaining data
            end = len(df)
        else:
            end = start + fold_size
        folds[i] = df[start:end]
    return folds

In [8]:
def find_index(lst, target):
    for i, number in enumerate(lst):
        if number == target:
            return i
    return None

In [9]:
def train_final_model(folds_X, folds_y):
    cv_amount = len(folds_X)
    models, rmse_scores, predictions = {}, {}, {}
    rmse_total = 0
    
    for i in range(cv_amount):
        test_X, test_y = folds_X[i], folds_y[i]
        train_X = np.concatenate([folds_X[n] for n in range(len(folds_X)) if n != i])
        train_y = np.concatenate([folds_y[n] for n in range(len(folds_y)) if n != i])

        models['model_{}'.format(i)] = train_model_statsmodels(train_X, train_y)
        test_X = sm.add_constant(test_X, has_constant='add')
        y_pred = models['model_{}'.format(i)].predict(test_X)
        predictions[i] = y_pred
        
        rmse_scores[i] = mean_squared_error(np.log1p(test_y), np.log1p(y_pred), squared=False)
        rmse_total += rmse_scores[i]

    # Final model and the final model's score
    rmse_average = rmse_total / cv_amount
    rmse_list = []
    for i in range(len(rmse_scores)):
        rmse_list.append(rmse_scores[i])
    max_rmse = max(rmse_list)
    min_rmse = min(rmse_list)
    index_max_rmse = find_index(rmse_list, max_rmse)
    index_min_rmse = find_index(rmse_list, min_rmse)
    final_model = models['model_{}'.format(index_max_rmse)]
    display(final_model.summary())
    
    # Residual analysis
    #residuals_cv = np.expm1(folds_y[index_min_r2]) - predictions[index_min_r2]
    #plot_residuals(residuals_cv, predictions[index_min_r2])
    #normality(residuals_cv)
    #print(residuals_cv.sort_values())
    print("-----------------------------------------------------------------------------")
    print("All RMSE score:  ", rmse_scores)
    print("Max RMSE score:  ", rmse_scores[index_max_rmse], index_max_rmse)
    print("Min RMSE score:  ", rmse_scores[index_min_rmse], index_min_rmse)
    print("Average RMSE score:  ", rmse_average)
    return final_model, rmse_average

In [10]:
def cyclic_features(df, df_):
    df['date_decision'] = pd.to_datetime(df['date_decision'])
    
    days_in_year = 365.25
    df['day_of_year'] = df['date_decision'].dt.dayofyear
    df['year_sin'] = np.sin(2 * np.pi * df['day_of_year'] / days_in_year)
    df['year_cos'] = np.cos(2 * np.pi * df['day_of_year'] / days_in_year)
    
    days_in_month = 30.437
    df['day_of_month'] = df['date_decision'].dt.day
    df['month_sin'] = np.sin(2 * np.pi * df['day_of_month'] / days_in_month)
    df['month_cos'] = np.cos(2 * np.pi * df['day_of_month'] / days_in_month)
    
    days_in_week = 7
    df['day_of_week'] = df['date_decision'].dt.dayofweek
    df['week_sin'] = np.sin(2 * np.pi * df['day_of_week'] / days_in_week)
    df['week_cos'] = np.cos(2 * np.pi * df['day_of_week'] / days_in_week)
    

    df_['date_decision'] = pd.to_datetime(df_['date_decision'])
    
    days_in_year = 365.25
    df_['day_of_year'] = df_['date_decision'].dt.dayofyear
    df_['year_sin'] = np.sin(2 * np.pi * df_['day_of_year'] / days_in_year)
    df_['year_cos'] = np.cos(2 * np.pi * df_['day_of_year'] / days_in_year)
    
    days_in_month = 30.437
    df_['day_of_month'] = df_['date_decision'].dt.day
    df_['month_sin'] = np.sin(2 * np.pi * df_['day_of_month'] / days_in_month)
    df_['month_cos'] = np.cos(2 * np.pi * df_['day_of_month'] / days_in_month)
    
    days_in_week = 7
    df_['day_of_week'] = df_['date_decision'].dt.dayofweek
    df_['week_sin'] = np.sin(2 * np.pi * df_['day_of_week'] / days_in_week)
    df_['week_cos'] = np.cos(2 * np.pi * df_['day_of_week'] / days_in_week)
    return df, df_

In [11]:
train_basetable = reduce_mem_usage(pd.read_csv(data + "train/train_base.csv"))
train_static = reduce_mem_usage(pd.concat(
    [
        set_table_dtypes(pd.read_csv(data + "train/train_static_0_0.csv")),
        set_table_dtypes(pd.read_csv(data + "train/train_static_0_1.csv"))
    ],
    axis=0
))
train_static_cb = reduce_mem_usage(set_table_dtypes(pd.read_csv(data + "train/train_static_cb_0.csv")))
train_person_1 = reduce_mem_usage(set_table_dtypes(pd.read_csv(data + "train/train_person_1.csv")))
train_credit_bureau_b_2 = reduce_mem_usage(set_table_dtypes(pd.read_csv(data + "train/train_credit_bureau_b_2.csv")))

Mem. usage decreased to 26.21 Mb (55.0% reduction)
Mem. usage decreased to 1952.41 Mb (0.3% reduction)
Mem. usage decreased to 601.01 Mb (0.9% reduction)
Mem. usage decreased to 808.32 Mb (3.7% reduction)
Mem. usage decreased to 36.81 Mb (37.5% reduction)


In [12]:
test_basetable = reduce_mem_usage(pd.read_csv(data + "test/test_base.csv"))
test_static = reduce_mem_usage(pd.concat(
    [
        set_table_dtypes(pd.read_csv(data + "test/test_static_0_0.csv")),
        set_table_dtypes(pd.read_csv(data + "test/test_static_0_1.csv"))
    ],
    axis=0
))
test_static_cb = reduce_mem_usage(set_table_dtypes(pd.read_csv(data + "test/test_static_cb_0.csv")))
test_person_1 = reduce_mem_usage(set_table_dtypes(pd.read_csv(data + "test/test_person_1.csv")))
test_credit_bureau_b_2 = reduce_mem_usage(set_table_dtypes(pd.read_csv(data + "test/test_credit_bureau_b_2.csv")))

Mem. usage decreased to  0.00 Mb (33.5% reduction)
Mem. usage decreased to  0.03 Mb (0.3% reduction)
Mem. usage decreased to  0.00 Mb (0.9% reduction)
Mem. usage decreased to  0.00 Mb (3.6% reduction)
Mem. usage decreased to  0.00 Mb (29.6% reduction)


In [13]:
# Aggregation in pandas for train_person_1
train_person_1_feats_1 = train_person_1.groupby('case_id').agg(
    mainoccupationinc_384A_max=('mainoccupationinc_384A', 'max'),
    mainoccupationinc_384A_any_selfemployed=('incometype_1044T', lambda x: (x == 'SELFEMPLOYED').max())
).reset_index()

# Filtering and renaming in pandas for train_person_1
train_person_1_feats_2 = train_person_1.loc[train_person_1['num_group1'] == 0, ['case_id', 'housetype_905L']]
train_person_1_feats_2 = train_person_1_feats_2.rename(columns={'housetype_905L': 'person_housetype'})

# Aggregation in pandas for train_credit_bureau_b_2
train_credit_bureau_b_2_feats = train_credit_bureau_b_2.groupby('case_id').agg(
    pmts_pmtsoverdue_635A_max=('pmts_pmtsoverdue_635A', 'max'),
    pmts_dpdvalue_108P_over31=('pmts_dpdvalue_108P', lambda x: (x > 31).max())
).reset_index()

# Process only A-type and M-type columns in pandas
selected_static_cols = [col for col in train_static.columns if col[-1] in ('A', 'M')]
print(selected_static_cols)

selected_static_cb_cols = [col for col in train_static_cb.columns if col[-1] in ('A', 'M')]
print(selected_static_cb_cols)

# Join all tables together using Pandas
train = reduce_mem_usage(train_basetable.merge(
    train_static[['case_id'] + selected_static_cols], on='case_id', how='left'
).merge(
    train_static_cb[['case_id'] + selected_static_cb_cols], on='case_id', how='left'
).merge(
    train_person_1_feats_1, on='case_id', how='left'
).merge(
    train_person_1_feats_2, on='case_id', how='left'
).merge(
    train_credit_bureau_b_2_feats, on='case_id', how='left'
))

['amtinstpaidbefduel24m_4187115A', 'annuity_780A', 'annuitynextmonth_57A', 'avginstallast24m_3658937A', 'avglnamtstart24m_4525187A', 'avgoutstandbalancel6m_4187114A', 'avgpmtlast12m_4525200A', 'credamount_770A', 'currdebt_22A', 'currdebtcredtyperange_828A', 'disbursedcredamount_1113A', 'downpmt_116A', 'inittransactionamount_650A', 'lastapprcommoditycat_1041M', 'lastapprcommoditytypec_5251766M', 'lastapprcredamount_781A', 'lastcancelreason_561M', 'lastotherinc_902A', 'lastotherlnsexpense_631A', 'lastrejectcommoditycat_161M', 'lastrejectcommodtypec_5251769M', 'lastrejectcredamount_222A', 'lastrejectreason_759M', 'lastrejectreasonclient_4145040M', 'maininc_215A', 'maxannuity_159A', 'maxannuity_4075009A', 'maxdebt4_972A', 'maxinstallast24m_3658928A', 'maxlnamtstart6m_4525199A', 'maxoutstandbalancel12m_4187113A', 'maxpmtlast3m_4525190A', 'previouscontdistrict_112M', 'price_1097A', 'sumoutstandtotal_3546847A', 'sumoutstandtotalest_4493215A', 'totaldebt_9A', 'totalsettled_863A', 'totinstallas

In [14]:
# Aggregation in pandas for train_person_1
test_person_1_feats_1 = test_person_1.groupby('case_id').agg(
    mainoccupationinc_384A_max=('mainoccupationinc_384A', 'max'),
    mainoccupationinc_384A_any_selfemployed=('incometype_1044T', lambda x: (x == 'SELFEMPLOYED').max())
).reset_index()

# Filtering and renaming in pandas for train_person_1
test_person_1_feats_2 = test_person_1.loc[test_person_1['num_group1'] == 0, ['case_id', 'housetype_905L']]
test_person_1_feats_2 = test_person_1_feats_2.rename(columns={'housetype_905L': 'person_housetype'})

# Aggregation in pandas for train_credit_bureau_b_2
test_credit_bureau_b_2_feats = test_credit_bureau_b_2.groupby('case_id').agg(
    pmts_pmtsoverdue_635A_max=('pmts_pmtsoverdue_635A', 'max'),
    pmts_dpdvalue_108P_over31=('pmts_dpdvalue_108P', lambda x: (x > 31).max())
).reset_index()

# Join all tables together using Pandas
test = reduce_mem_usage(test_basetable.merge(
    test_static[['case_id'] + selected_static_cols], on='case_id', how='left'
).merge(
    test_static_cb[['case_id'] + selected_static_cb_cols], on='case_id', how='left'
).merge(
    test_person_1_feats_1, on='case_id', how='left'
).merge(
    test_person_1_feats_2, on='case_id', how='left'
).merge(
    test_credit_bureau_b_2_feats, on='case_id', how='left'
))

Mem. usage decreased to  0.00 Mb (0.0% reduction)


In [15]:
train, test = cyclic_features(train, test)

#train = pd.get_dummies(train)
#test = pd.get_dummies(test)

In [16]:
train = train.apply(pd.to_numeric, errors='coerce')
test = test.apply(pd.to_numeric, errors='coerce')

selected_columns = [col for col in train.columns if col not in ['target', 'date_decision', 'day_of_year',
                                                                'day_of_month', 'day_of_week']]

imputer = SimpleImputer(strategy='constant', fill_value=0)
train[selected_columns] = imputer.fit_transform(train[selected_columns])

test[selected_columns] = preprocess_data(test[selected_columns], imputer)

y = train['target']
X = train[selected_columns]
X_test = test[selected_columns]

folds_X = create_k_folds(X, fold_n=5)
folds_y = create_k_folds(y, fold_n=5)

In [17]:
final_model, rmse_average = train_final_model(folds_X, folds_y)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     250.1
Date:                Tue, 14 May 2024   Prob (F-statistic):               0.00
Time:                        11:29:04   Log-Likelihood:             4.3698e+05
No. Observations:             1221328   AIC:                        -8.739e+05
Df Residuals:                 1221279   BIC:                        -8.733e+05
Df Model:                          48                                         
Covariance Type:                  HC0                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         14.0506      2.367      5.936      0.000       9.412      18.690
x1          3.001e-09   2.38e-10     12.604      0.000    2.53e-09    3.47e-09
x2         -6.946e-05   1.17e-05     -5.924      0.000   -9.24e-05   -4.65e-05
x3          5.064e-05   2.26e-05      2.237      0.025    6.28e-06     9.5e-05
x4         -1.905e-07   7.44e-09    -25.589      0.000   -2.05e-07   -1.76e-07
x5         -6.084e-07   9.45e-08     -6.436      0.000   -7.94e-07   -4.23e-07
x6          2.938e-08   8.94e-08      0.329      0.743   -1.46e-07    2.05e-07
x7         -1.656e-07   4.19e-08     -3.953      0.000   -2.48e-07   -8.35e-08
x8          4.527e-08   1.39e-08      3.248      0.001     1.8e-08    7.26e-08
x9         -2.545e-08   1.86e-08     -1.372      0.170   -6.18e-08    1.09e-08
x10         1.804e-07   4.18e-08      4.318      0.000    9.85e-08    2.62e-07
x11         9.862e-08    1.1e-08      8.961      0.000     7.7e-08     1.2e-07
x12        -4.196e-06   2.18e-06     -1.924      0.054   -8.47e-06    7.94e-08
x13         2.567e-08   7.42e-09      3.461      0.001    1.11e-08    4.02e-08
x14         6.209e-08   1.08e-08      5.773      0.000     4.1e-08    8.32e-08
x15         5.342e-08   4.93e-08      1.083      0.279   -4.33e-08     1.5e-07
x16         -1.15e-07   1.36e-08     -8.465      0.000   -1.42e-07   -8.83e-08
x17        -3.305e-18    1.6e-18     -2.071      0.038   -6.43e-18   -1.77e-19
x18        -6.503e-18   1.64e-18     -3.955      0.000   -9.73e-18   -3.28e-18
x19        -4.652e-08   6.44e-09     -7.220      0.000   -5.91e-08   -3.39e-08
x20         1.052e-17   9.45e-19     11.123      0.000    8.66e-18    1.24e-17
x21        -2.106e-07   5.79e-07     -0.364      0.716   -1.35e-06    9.24e-07
x22         -3.92e-08   1.28e-07     -0.307      0.759   -2.89e-07    2.11e-07
x23          5.74e-17   4.44e-18     12.941      0.000    4.87e-17    6.61e-17
x24        -3.383e-17   3.97e-18     -8.513      0.000   -4.16e-17    -2.6e-17
x25           1.4e-07   4.19e-09     33.450      0.000    1.32e-07    1.48e-07
x26        -1.431e-17   1.56e-18     -9.172      0.000   -1.74e-17   -1.13e-17
x27         1.918e-17   1.09e-18     17.608      0.000     1.7e-17    2.13e-17
x28         9.458e-09    5.8e-09      1.632      0.103    -1.9e-09    2.08e-08
x29         4.307e-08   5.07e-09      8.502      0.000    3.31e-08     5.3e-08
x30         2.618e-08   6.19e-09      4.230      0.000    1.41e-08    3.83e-08
x31        -6.666e-08   8.35e-09     -7.986      0.000    -8.3e-08   -5.03e-08
x32         1.693e-07   1.63e-08     10.374      0.000    1.37e-07    2.01e-07
x33        -2.355e-08   1.01e-08     -2.328      0.020   -4.34e-08   -3.72e-09
x34         1.466e-07   1.19e-08     12.302      0.000    1.23e-07     1.7e-07
x35         2.225e-08   3.29e-08      0.675      0.499   -4.23e-08    8.68e-08
x3

-----------------------------------------------------------------------------
All RMSE score:   {0: 0.12345410439223918, 1: 0.11832400229556526, 2: 0.1126427008321152, 3: 0.13157326835440308, 4: 0.118335398990667}
Max RMSE score:   0.13157326835440308 3
Min RMSE score:   0.1126427008321152 2
Average RMSE score:   0.12086589497299796


In [18]:
X = np.asarray(X)
X_test = np.asarray(X_test)

In [19]:
model = train_model_statsmodels(X, y)

X = sm.add_constant(X)
y_pred = model.predict(X)
rmse_score = mean_squared_error(np.log1p(y), np.log1p(y_pred), squared=False)
print("RMSE", rmse_score)

RMSE 0.12072608383954016


In [20]:
X_test = sm.add_constant(X_test, has_constant='add')
predicted_log_score = model.predict(X_test)
predicted_score = np.expm1(predicted_log_score)

In [21]:
predicted_score_df = pd.DataFrame({
    'case_id': test['case_id'].to_numpy(),
    'score': predicted_score
})
predicted_score_df.to_csv('submission.csv', index=False)

In [22]:
predicted_score_df

,case_id,score
0,57543.0,0.027529
1,57549.0,0.054640
2,57551.0,0.011501
3,57552.0,0.048497
4,57569.0,0.019284
5,57630.0,0.042133
6,57631.0,0.026451
7,57632.0,-0.010984
8,57633.0,0.067623
9,57634.0,0.022384
